In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from sklearn.neighbors import NearestNeighbors
import numpy as np
import os
import sys
import webbrowser
from time import time_ns

sys.path.append(os.path.join(os.path.dirname(os.getcwd())))
from interfaces.alchemy import AlchemyInterface
import configparser
from db.schema import StationLocation
import geopandas as gpd
from shapely.geometry import Point

In [2]:
c_parser = configparser.ConfigParser()
c_parser.read("../../config.ini")
config = {"db": dict(c_parser["db"])}

In [3]:
alchemy_interface = AlchemyInterface(config)

In [4]:
time_start = time_ns()
data = alchemy_interface.select_obj(
    StationLocation,
    **{"station": "W6HBR-2"},
    df=False,
)
time_query = time_ns()
print("Time to query data: ", (time_query - time_start) / 1e6, " ms")

Query: SELECT station, timestamp, latitude, longitude, country 
FROM aprsint.station_locations 
WHERE station = 'W6HBR-2'
Time to query data:  52.277713  ms


In [ ]:
df = df.dropna(subset=["latitude", "longitude"])

In [ ]:
coords = df[["latitude", "longitude"]].values

# Usar NearestNeighbors para encontrar vecinos en un radio
# El radio está en grados y puedes ajustarlo según tus necesidades
radius = 0.5  # Ajusta este valor según tus necesidades
nbrs = NearestNeighbors(radius=radius, algorithm="ball_tree").fit(coords)
distances, indices = nbrs.radius_neighbors(coords)

# Calcular la densidad: número de puntos en el radio para cada punto
density = np.array([len(indices[i]) for i in range(len(coords))])

# Agregar la densidad al DataFrame original
df["density"] = density

In [ ]:
def do_click(trace, points, state):
    print("clicked")
    if points.point_inds:
        ind = points.point_inds[0]
        url = df.link.iloc[ind]
        webbrowser.open_new_tab(url)


# The logic of drawing a graph
def bar_func(df):
    fig = px.scatter_mapbox(
        df,
        lon="longitude",
        lat="latitude",
    )

    fig.update_traces(
        marker=go.scattermapbox.Marker(
            size=5,
        ),
    )

    fig.update_layout(
        hovermode="closest",
        mapbox_style="dark",
        mapbox_accesstoken=c_parser["mapbox"]["token"],
    )
    fig.data[0].on_click(do_click)
    fig.show()


bar_func(df)

In [ ]:
x = np.random.rand(100)
y = np.random.rand(100)

f = go.FigureWidget([go.Scatter(x=x, y=y, mode="markers")])
scatter = f.data[0]
f.layout.hovermode = "closest"


def do_click(trace, points, state):
    print("Clicked")
    if points.point_inds:
        ind = points.point_inds[0]
        url = df.link.iloc[ind]
        webbrowser.open_new_tab(url)


scatter.on_click(do_click)
f

In [ ]:
import plotly.graph_objects as go

import numpy as np

np.random.seed(1)

x = np.random.rand(100)
y = np.random.rand(100)

f = go.FigureWidget([go.Scatter(x=x, y=y, mode="markers")])

scatter = f.data[0]
colors = ["#a3a7e4"] * 100
scatter.marker.color = colors
scatter.marker.size = [10] * 100
f.layout.hovermode = "closest"


# create our callback function
def update_point(trace, points, selector):
    print("Hemlo")
    # c = list(scatter.marker.color)
    # s = list(scatter.marker.size)
    # for i in points.point_inds:
    #     c[i] = '#bae2be'
    #     s[i] = 20
    #     with f.batch_update():
    #         scatter.marker.color = c
    #         scatter.marker.size = s


scatter.on_click(update_point)

f